In [624]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import datetime
from IPython.display import display, HTML
from termcolor import colored

In [625]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [626]:
# df_weekly = pd.read_csv('../data/clean/df_weekly.csv')
df_weekly = pd.read_csv('../data/clean/df_weekly_incidence.csv')

In [627]:
data_admin_area = df_weekly.administrative_area_level_3.values

In [628]:
df_weekly.week.max()
lk_year_week = df_weekly[["administrative_area_level_3","year","week"]]
years = lk_year_week.year.unique()

min_data_per_lk = 99999999
max_data_per_lk = 0
data_per_lk_list = []
for lk in lk_year_week.administrative_area_level_3.unique():
    landk_len = len(lk_year_week[lk_year_week.administrative_area_level_3 == lk])
    # week_sum = 0
    # for y in years:
    #     week_sum += landk[lk_year_week.year == y].week.max()
    if landk_len < min_data_per_lk:
        min_data_per_lk = landk_len
    if landk_len > min_data_per_lk:
        max_data_per_lk = landk_len

    data_per_lk_list.append((lk,landk_len))
    #     min_weeks = week_sum
    # if week_sum > max_weeks:
    #     max_weeks = week_sum
# print(min_weeks,max_weeks)
print(min_data_per_lk,max_data_per_lk)

100 148


In [629]:
'''
Teile die In-Seqeuenz (Feature) in einzelne Sequenzen der Länge n_steps_in und
die Out-Sequenz (Target) in einzelne Sequenzen der Länge n_steps_out.

Beispiel: n_steps_in=1, n_steps_out=2
           Feature  | Target
    Seq 1:    1         -
        |     2         -
        |     3         6
        |     -         8       

@param overap 
Sollen sich die einzelnen Sequenzen überlappen?
    Standardmäßig überlappen sich die Daten der Sequenzen:
        1. Seq [1,2,3]
        2. Seq [2,3,4]
    mit overlap = False:
        1. Seq [1,2,3]
        2. Seq [4,5,6]

'''
def split_sequence(in_sequence, out_sequence, n_steps_in, n_steps_out, overlap=True, last_feature_as_target=True):
    X, y = list(), list()
    step = 1 if overlap else n_steps_in
    left_over = 0 
    target_offset = 1 if last_feature_as_target else 0

    for i in range(0,len(in_sequence),step):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - target_offset
        # check if we are beyond the sequence
        if (out_end_ix > len(in_sequence)):
            left_over = len(in_sequence) - i
            break
   
        # gather input and output parts of the pattern     
        seq_x, seq_y = in_sequence[i:end_ix,:],  out_sequence[end_ix-target_offset:out_end_ix,-1] #out_sequence[end_ix-1:out_end_ix,-1]
        X.append(seq_x)
        y.append(seq_y)

       
    return np.array(X), np.array(y), left_over

In [630]:
'''
Prüfe, ob in einer Sequenzen basieren auf den IN und OUT Steps 
LK übergreifenden Daten vorkommen.
Entferne Zeilen, welche nicht mehr für eine volle Sequenz verwendet werden können

BSP:
    1. LK Erfurt
    2. LK Erfurt
    3. LK Erfurt
    4. LK Ahrweiler
    5. LK Ahrweiler

'''
def prepare_df_for_seqeuncing(df,n_steps_in, n_steps_out):
    droped_indices = []
    df_len = len(df)-1
    for i in range(0,df_len):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out -1
        # check if we are beyond the sequence
        if (out_end_ix > df_len):
            break
        start_row = df.loc[i]
        end_row = df.loc[out_end_ix:out_end_ix]

        lk_of_seq_start = start_row.administrative_area_level_3
        lk_of_seq_end = end_row.administrative_area_level_3.iloc[0]

        # Wenn es nicht mehr genug "Zeilen" eines LKs gibt müssen diese entfernt werden
        if (lk_of_seq_start != lk_of_seq_end):
            droped_indices.append(i)          
    
    df = df.drop(df.index[droped_indices])
    return df

In [715]:

TIME_STEPS_IN = 20
TIME_STEPS_OUT = 10
DATA_OVERLAP = True
# TARGET_COLUMN = "incidence"
TARGET_COLUMN = ""
TARGET_COLUMN = "deaths"
TRAIN_DATA_COLUMNS = None
TARGET_COLUMN_INDEX = 0

print("DF Full:",len(df_weekly))

df_weekly_adjusted = prepare_df_for_seqeuncing(df_weekly,TIME_STEPS_IN, TIME_STEPS_OUT)

data = df_weekly_adjusted[['confirmed', 'deaths' ,"recovered"
    ,"vaccines","people_vaccinated","people_fully_vaccinated"
    ,"school_closing","workplace_closing","cancel_events"
    ,"gatherings_restrictions","transport_closing"
    ,"stay_home_restrictions","internal_movement_restrictions"
    ,"internal_movement_restrictions","information_campaigns","testing_policy"
    ,"contact_tracing","facial_coverings"
    ,"vaccination_policy","elderly_people_protection","population","cfr","cases_per_population","incidence"]]

TRAIN_DATA_COLUMNS = data.columns
TARGET_COLUMN_INDEX = TRAIN_DATA_COLUMNS.get_loc(TARGET_COLUMN)
 

data = np.array(data)

print("Data:",data.shape)

# new_data = np.array([[39919.42857142857, 1132.2857142857142, 38787.142857142855, 8832.0, 8832.0, 0.0, 3.0, 3.0, 2.0, 4.0, 1.0, 1.4285714285714286, -1.5714285714285714,
#                       3.0, 2.0, 1.0, 1.1428571428571428, 2.0, 2.0, 2.0, 1841179.0]])  # Beispielhafte Eingangsdaten (Features)
# Vorbereitung der Eingangsdaten (Features) und der Zielvariablen
features = data[:, :] 
target = data[:, -1]

## Nimm Deaths als Target
# target = data[:, 1]
# # features = data[:,:3] ## nim erstmal nur confirmed deaths
# features = np.delete(data,1,axis=1)[:,:2]
print("Features:",features.shape,"Target:",target.shape)

N_FEATURES = features.shape[-1]


# Daten normalisieren
feature_scaler = MinMaxScaler(feature_range=(0, 1))
target_scaler = MinMaxScaler(feature_range=(0, 1))

features_scaled = feature_scaler.fit_transform(features)
target_scaled = target_scaler.fit_transform(target.reshape(-1, 1))
print("Features Scaled:",features_scaled.shape,"Target Scaled:",target_scaled.shape)
## SCALING Muss nach sequenzierung passieren, sonst passt die Normaliseirung nicht, wenn Werte fehlen

feature_seqs, target_seq, left_over =  split_sequence(features_scaled,target_scaled, TIME_STEPS_IN, TIME_STEPS_OUT, overlap=DATA_OVERLAP,last_feature_as_target=False)
df_weekly_adjusted = df_weekly_adjusted.drop(df_weekly_adjusted.index[-left_over:])
print("Features Seq.:",feature_seqs.shape,"Target Seq.:",target_seq.shape, "DF Adjust.:",len(df_weekly_adjusted))




train_size = int(len(feature_seqs) * 0.8)
test_size = len(feature_seqs) - train_size
print("Train Size: {}, Test Size: {}".format(train_size,test_size))
# train_features, test_features = features_scaled[:train_size], features_scaled[train_size:]
# train_target, test_target = target_scaled[:train_size], target_scaled[train_size:]


# feature -> target
# t1 -> t2
### ALTERNATIVE: Keras TimeSeriesGenerator https://keras.io/api/data_loading/timeseries/ 
# def create_lstm_dataset(features, target, time_steps):
#     X, y = [], []
#     for i in range(len(features) - time_steps):
#         X.append(features[i:(i+time_steps)])
#         y.append(target[i+time_steps])
#     return np.array(X), np.array(y)

# Das problem mit der input Größe >.<. Aktuell werden alle 5 daten aus den train data aneinandergereiht. Hier muss noch eine lösung her

# train_features_lstm, train_target_lstm = create_lstm_dataset(train_features, train_target, TIME_STEPS_IN)
# test_features_lstm, test_target_lstm = create_lstm_dataset(test_features, test_target, TIME_STEPS_IN)






DF Full: 58920
Data: (47001, 24)
Features: (47001, 24) Target: (47001,)
Features Scaled: (47001, 24) Target Scaled: (47001, 1)
Features Seq.: (46972, 20, 24) Target Seq.: (46972, 10) DF Adjust.: 46972
Train Size: 37577, Test Size: 9395


In [716]:
# X, y = split_sequence(features_scaled,target_scaled, TIME_STEPS_IN, TIME_STEPS_OUT, admin_area=data_admin_area, overlap=True)
# print(X.shape,y.shape)

train_features_lstm, test_features_lstm = feature_seqs[:train_size], feature_seqs[train_size:]
train_target_lstm, test_target_lstm = target_seq[:train_size], target_seq[train_size:]

print("TRAIN:",train_features_lstm.shape,"TEST:",test_features_lstm.shape)
print(train_features_lstm[0,:,:2],train_target_lstm[0,:])
# print(train_features_lstm[1,:,:2],train_target_lstm[1,:])

TRAIN: (37577, 20, 24) TEST: (9395, 20, 24)
[[1.26129814e-05 0.00000000e+00]
 [5.40556346e-05 0.00000000e+00]
 [9.85286793e-05 2.55937756e-04]
 [1.40053235e-04 3.58312858e-04]
 [1.80881620e-04 3.58312858e-04]
 [1.94354577e-04 3.58312858e-04]
 [1.99514433e-04 3.58312858e-04]
 [2.17401934e-04 3.58312858e-04]
 [2.45609147e-04 3.58312858e-04]
 [2.75765639e-04 3.58312858e-04]
 [3.10574191e-04 3.58312858e-04]
 [3.22204343e-04 3.58312858e-04]
 [3.30804103e-04 3.58312858e-04]
 [3.36250618e-04 3.58312858e-04]
 [3.38830546e-04 3.58312858e-04]
 [3.53880126e-04 3.58312858e-04]
 [3.68069730e-04 3.58312858e-04]
 [3.75522855e-04 4.77750478e-04]
 [3.83549298e-04 7.16625717e-04]
 [3.87849178e-04 7.16625717e-04]] [0.00169892 0.00240448 0.00373669 0.003927   0.00539382 0.00337926
 0.0013647  0.00199831 0.00220952 0.00487394]


In [633]:
# model = Sequential()
# model.add(tf.keras.layers.Bidirectional(LSTM(4, activation='relu', 
#                 # return_sequences=True,
#                 input_shape=(TIME_STEPS_IN, N_FEATURES))))
# # model.add(tf.keras.layers.Dropout(0.2))
# # model.add(LSTM(2, activation='relu'))
# # model.add(tf.keras.layers.Dropout(0.2))
# # model.add(Dense(128))
# model.add(Dense(TIME_STEPS_OUT))
# model.compile(optimizer='adam', loss='mse')

In [740]:
''' 
Encoder-Decoder Architecture für Sequence to Sequence
https://machinelearningmastery.com/encoder-decoder-long-short-term-memory-networks/

'''

# activ = 'tanh'
activ = 'relu'
# loss = tf.keras.losses.MeanSquaredError()
loss = "mse"
# loss = tf.keras.losses.CategoricalCrossentropy()
optim = tf.keras.optimizers.Adam()

ndim = 500

model = Sequential()
model.add(LSTM(ndim, activation=activ, input_shape=(TIME_STEPS_IN, N_FEATURES)))
model.add(tf.keras.layers.RepeatVector(TIME_STEPS_OUT)) # damit der Output 3d ([samples, time steps, features]) 
model.add(LSTM(ndim, activation=activ, return_sequences=True)) # return_sequences: Gibt den gesamten Output Eingabesequenz zurück und nicht nur den letzen
model.add(LSTM(100, activation=activ, return_sequences=True)) # return_sequences: Gibt den gesamten Output Eingabesequenz zurück und nicht nur den letzen
model.add(tf.keras.layers.TimeDistributed(Dense(1)))
model.compile(optimizer=optim, loss=loss,metrics=['accuracy'])
model.summary()

Model: "sequential_83"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_201 (LSTM)             (None, 500)               1050000   
                                                                 
 repeat_vector_56 (RepeatVec  (None, 10, 500)          0         
 tor)                                                            
                                                                 
 lstm_202 (LSTM)             (None, 10, 500)           2002000   
                                                                 
 lstm_203 (LSTM)             (None, 10, 100)           240400    
                                                                 
 time_distributed_56 (TimeDi  (None, 10, 1)            101       
 stributed)                                                      
                                                                 
Total params: 3,292,501
Trainable params: 3,292,501
N

In [635]:
# returns train, inference_encoder and inference_decoder models
def define_models(n_input, n_output, n_units):
    # define training encoder
    # encoder_inputs = tf.keras.layers.Input(shape=(None, n_input))
    encoder_inputs = tf.keras.layers.Input(shape=(None, n_input))
    encoder = LSTM(n_units, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)
    encoder_states = [state_h, state_c]
    # define training decoder
    decoder_inputs = tf.keras.layers.Input(shape=(None, n_output))
    decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)
    model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
    # define inference encoder
    encoder_model = tf.keras.Model(encoder_inputs, encoder_states)
    # define inference decoder
    decoder_state_input_h = tf.keras.layers.Input(shape=(n_units,))
    decoder_state_input_c = tf.keras.layers.Input(shape=(n_units,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
    # return all models
    return model, encoder_model, decoder_model

In [636]:
# model, inference_encoder, inference_decoder = define_models(TIME_STEPS_IN,TIME_STEPS_OUT,20)
# model.compile(optimizer='adam', loss='mse',metrics=['accuracy'])
# model.summary()

In [741]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


class haltCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        stop_at = 0.03
        if(logs.get('loss') <= stop_at):
            print(f"\n\n\nReached {stop_at} loss value so cancelling training!\n\n\n")
            self.model.stop_training = True


## Trainiere                                                
history = model.fit(
                    train_features_lstm, 
                    train_target_lstm, 
                    epochs=3, 
                    batch_size=32, 
                    validation_data=(test_features_lstm, test_target_lstm),
                    callbacks=[
                        tensorboard_callback,
                        # haltCallback()
                        tf.keras.callbacks.EarlyStopping(monitor='loss', patience=200),

                        ])


loss = model.evaluate(test_features_lstm, test_target_lstm)


print('Test Loss:', loss)

Epoch 1/3
1175/1175 [==============================] - 99s 82ms/step - loss: 0.0033 - accuracy: 0.0107 - val_loss: 0.0023 - val_accuracy: 0.0175
Epoch 2/3
1175/1175 [==============================] - 96s 82ms/step - loss: 0.0020 - accuracy: 0.0107 - val_loss: 0.0023 - val_accuracy: 0.0172
Epoch 3/3
294/294 [==============================] - 7s 24ms/step - loss: 0.0021 - accuracy: 0.0171
Test Loss: [0.0020886054262518883, 0.01709420047700405]


In [654]:

model.save("model.h5")

In [739]:
INDEX = 30
TEST_INDEX_L = INDEX 
TEST_INDEX_H = INDEX+1
val_features = test_features_lstm[TEST_INDEX_L:TEST_INDEX_H]
val_target = test_target_lstm[TEST_INDEX_L:TEST_INDEX_H]

prediction = model.predict(val_features)
# prediction



## * IN Sequence  | Time Steps die nur reingegeben werden
## * OUT Sequence | Time Steps die vom Netzwerk predicted werden
## * TIME_STEPS_OUT, da die Prediction für zwei Zeitschritte aus der Prediction des letzen Val-Wertes und des ersten komplett unbekannten besteht.
## -> ÜBerscheiden sich am Ende 
start_test_data = train_size
start_val_data = start_test_data +TEST_INDEX_L
end_val_data = (TEST_INDEX_H-TEST_INDEX_L)*TIME_STEPS_IN
end_prediciton_vals= TIME_STEPS_OUT -1




## zur Kontrolle: de-scalierte Features
de_time_series_val_fatures = val_features.reshape((TEST_INDEX_H-TEST_INDEX_L)*TIME_STEPS_IN,N_FEATURES)
de_scaled_val_features = feature_scaler.inverse_transform(de_time_series_val_fatures)

de_scaled_target_features = target_scaler.inverse_transform(val_target.reshape(-1,1)).reshape(-1)
de_scaled_prediction = target_scaler.inverse_transform(prediction.reshape(-1,1)).reshape(-1)

df_weekly_val = df_weekly_adjusted.iloc[start_val_data:start_val_data+end_val_data]

except_taget_rows = df_weekly_adjusted.iloc[start_val_data+end_val_data-1:start_val_data+end_val_data+end_prediciton_vals]

print(f"Prediction for: {TARGET_COLUMN}")
values_is = except_taget_rows[TARGET_COLUMN].to_numpy()

result_df = pd.DataFrame(data={
    "Row":except_taget_rows.index.values,
    "Expected":de_scaled_prediction,
    "Predictied":values_is,
    "abs(Diff)":abs(de_scaled_prediction -  values_is)})

display(result_df)

print("Val Expected Target")
display(except_taget_rows)



print("Kontrolle: de-skalierte Features:")
print(" * Zeile {} | confirmed: {:.2f}, recovered: {:.2f}".format(df_weekly_val.index.values[0],de_scaled_val_features[0,0],de_scaled_val_features[0,1]))
print("\nVal Input Features:")
display(df_weekly_val)





1/1 [==============================] - 0s 213ms/step
Prediction for: deaths


,Row,Expected,Predictied,abs(Diff)
0,47167,91.940948,491.857143,399.916194
1,47168,92.892960,505.428571,412.535612
2,47169,123.446556,522.428571,398.982015
3,47170,171.778976,531.857143,360.078166
4,47171,219.964645,540.428571,320.463926
5,47172,255.604950,550.285714,294.680764
6,47173,266.347809,569.714286,303.366477
7,47174,252.403442,591.857143,339.453700
8,47175,220.394943,603.571429,383.176485
9,47176,187.221359,620.000000,432.778641


Val Expected Target


,administrative_area_level_3,year,week,confirmed,deaths,recovered,vaccines,people_vaccinated,people_fully_vaccinated,school_closing,...,contact_tracing,facial_coverings,vaccination_policy,elderly_people_protection,latitude,longitude,population,cfr,cases_per_population,incidence
47167,SK Düsseldorf,2021,41,32575.142857,491.857143,32083.285714,9.728964e+05,499179.428571,464496.714286,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.015099,0.052600,63.943781
47168,SK Düsseldorf,2021,42,33122.142857,505.428571,32616.714286,9.813856e+05,501920.428571,467733.857143,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.015259,0.053484,88.326385
47169,SK Düsseldorf,2021,43,33913.142857,522.428571,33390.714286,9.915246e+05,504817.000000,471129.857143,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.015405,0.054761,127.726088
47170,SK Düsseldorf,2021,44,34714.428571,531.857143,34182.571429,1.002156e+06,507766.714286,473856.428571,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.015322,0.056055,129.386966
47171,SK Düsseldorf,2021,45,35883.428571,540.428571,35343.000000,1.018726e+06,512299.571429,476626.000000,-1.142857,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.015062,0.057942,188.763334
47172,SK Düsseldorf,2021,46,37431.571429,550.285714,36881.285714,1.040547e+06,517989.285714,479139.428571,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.014704,0.060442,249.985121
47173,SK Düsseldorf,2021,47,39426.571429,569.714286,38856.857143,1.079378e+06,525601.857143,482678.285714,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.014450,0.063664,322.141019
47174,SK Düsseldorf,2021,48,41348.571429,591.857143,40756.714286,1.132190e+06,531638.571429,486626.571429,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.014315,0.066767,310.353402
47175,SK Düsseldorf,2021,49,43319.428571,603.571429,42715.857143,1.193956e+06,536012.000000,490758.571429,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.013934,0.069950,318.242570
47176,SK Düsseldorf,2021,50,44949.285714,620.000000,44329.285714,1.258602e+06,540214.571429,495722.428571,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.013794,0.072581,263.179870


Kontrolle: de-skalierte Features:
 * Zeile 47148 | confirmed: 24888.57, recovered: 445.00

Val Input Features:


,administrative_area_level_3,year,week,confirmed,deaths,recovered,vaccines,people_vaccinated,people_fully_vaccinated,school_closing,...,contact_tracing,facial_coverings,vaccination_policy,elderly_people_protection,latitude,longitude,population,cfr,cases_per_population,incidence
47148,SK Düsseldorf,2021,22,24888.571429,445.000000,24443.571429,431823.000000,302992.571429,128830.428571,1.000000,...,2.0,2.0,3.0,2.0,51.237923,6.78603,619294.0,0.017880,0.040189,33.748107
47149,SK Düsseldorf,2021,23,25022.857143,445.000000,24577.857143,483351.428571,317965.571429,165383.857143,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017784,0.040405,21.683678
47150,SK Düsseldorf,2021,24,25092.285714,445.000000,24647.285714,535976.142857,335177.142857,200772.428571,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017735,0.040518,11.210923
47151,SK Düsseldorf,2021,25,25133.285714,445.000000,24688.285714,586443.285714,355435.285714,230963.857143,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017706,0.040584,6.620442
47152,SK Düsseldorf,2021,26,25195.285714,445.000000,24750.285714,639009.428571,379167.285714,259785.428571,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017662,0.040684,10.011400
47153,SK Düsseldorf,2021,27,25296.857143,445.428571,24851.428571,686465.000000,400080.857143,286318.142857,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017608,0.040848,16.401165
47154,SK Düsseldorf,2021,28,25469.285714,446.285714,25023.000000,729273.428571,418491.714286,310705.142857,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017523,0.041126,27.842765
47155,SK Düsseldorf,2021,29,25699.428571,447.000000,25252.428571,759918.142857,429255.285714,330580.000000,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017394,0.041498,37.162133
47156,SK Düsseldorf,2021,30,25954.428571,447.000000,25507.428571,785616.000000,436600.000000,348908.857143,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017223,0.041910,41.175920
47157,SK Düsseldorf,2021,31,26228.285714,447.714286,25780.571429,810197.428571,443181.142857,366889.000000,1.000000,...,2.0,2.0,5.0,2.0,51.237923,6.78603,619294.0,0.017070,0.042352,44.220862


In [658]:



def coloring(string,color="normal",style="normal"):
    col = "0"
    sty = "2"
    if color == "blink":
        col = "5"
    elif   color == "red":
        col = "31"
    elif color == "green":
        col = "32"
    elif color == "yellow":
        col = "33"
    elif color == "blue":
        col = "34"
    elif color == "purple":
        col = "35"

    if style == "bold":
        sty = "1"
    elif style == "italic":
        sty = "3"
    elif style == "curl":
        sty = "4"
    elif style == "blink1":
        sty = "5"
    elif style == "blink2":
        sty = "6"
    elif style == "selected":
        sty = "7"

    return "\x1b[{}m\x1b[{}m{}\x1b[0m".format(col,sty,string)




print("Prediction for: {}".format(coloring(TARGET_COLUMN,"yellow")))

column_string = "    {}    | {}  | {} |  {}".format(coloring("Row","blue","bold"),coloring("Expected","green","bold"),coloring("Predicted","purple","bold"),coloring("abs(Diff)","red","bold"))
print("{:#<50s}\n{}\n{:-<50s}".format("",column_string,""))
for i, row in enumerate(except_taget_rows.index.values):
    values_predict = de_scaled_prediction[i]
    values_is = except_taget_rows.loc[row][TARGET_COLUMN]
    # print("Zeile {} | {} (Predicted): {:.1f} , {} (Is): {:.1f}, |Diff|: {:.2f} ".format(row,TARGET_COLUMN,values_predict, TARGET_COLUMN ,values_is, abs(values_predict-values_is))) 
    diff = abs(values_predict-values_is)
    print(" {: ^22} | {: ^22} | {: ^22} | {: ^22}".format(coloring(row,"blue"),coloring(f"{values_predict:.1f}","green"),coloring(f"{values_is:.1f}","purple"),coloring(f"{diff:.1f}","red"))) 




Prediction for: deaths
##################################################
    Row    | Expected  | Predicted |  abs(Diff)
--------------------------------------------------
   47167   |   92.1    |   491.9   |   399.8  
   47168   |   108.7   |   505.4   |   396.8  
   47169   |   164.2   |   522.4   |   358.2  
   47170   |   192.2   |   531.9   |   339.6  
   47171   |   199.3   |   540.4   |   341.1  
   47172   |   198.6   |   550.3   |   351.6  
   47173   |   196.0   |   569.7   |   373.7  
   47174   |   193.7   |   591.9   |   398.2  
   47175   |   186.7   |   603.6   |   416.8  
   47176   |   176.0   |   620.0   |   444.0  


In [659]:

tmp_feature_scaler = MinMaxScaler(feature_range=(0, 1))
tmp_target_scaler = MinMaxScaler(feature_range=(0, 1))
tmp_feat = features[:TIME_STEPS_IN,:]
tmp_targ = target[TIME_STEPS_IN:TIME_STEPS_IN+TIME_STEPS_OUT].reshape(TIME_STEPS_OUT,1)

tmp_feat_sc = tmp_feature_scaler.fit_transform(tmp_feat).reshape(1,TIME_STEPS_IN,len(TRAIN_DATA_COLUMNS)-1)
tmp_targ_sc = tmp_target_scaler.fit_transform(tmp_targ)
tmp_predict_sc = model.predict(tmp_feat_sc)

tmp_predict_de_sc = tmp_target_scaler.inverse_transform(tmp_predict_sc[0,:,:])
tmp_predict_de_sc

1/1 [==============================] - 0s 14ms/step


array([[8.954931 ],
       [7.697418 ],
       [6.398279 ],
       [7.173036 ],
       [7.5840364],
       [7.192413 ],
       [6.987025 ],
       [7.1187477],
       [7.137819 ],
       [7.019142 ]], dtype=float32)

In [714]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf


'''
 *** Infos ***

Input Data Columns:

    'confirmed', 'deaths', 'recovered', 'vaccines', 'people_vaccinated','people_fully_vaccinated', 'school_closing', 'workplace_closing',
    'cancel_events', 'gatherings_restrictions', 'transport_closing', 'stay_home_restrictions', 'internal_movement_restrictions',
    'internal_movement_restrictions', 'information_campaigns', 'testing_policy', 'contact_tracing', 'facial_coverings','vaccination_policy',
    'elderly_people_protection', 'population', 'cfr', 'cases_per_population', 'incidence'


Eingabe: numpy.array
    Shape:  (SEQUENCE_COUNT, INPUT_STEPS, INPUT_COLS)  

Ausgabe: numpy.array
    Shape:  (SEQUENCE_COUNT, OUTPUT_STEPS, 1)
   
'''


def get_relevant_cols(df):
    return df[['confirmed', 'deaths' ,"recovered"
        ,"vaccines","people_vaccinated","people_fully_vaccinated"
        ,"school_closing","workplace_closing","cancel_events"
        ,"gatherings_restrictions","transport_closing"
        ,"stay_home_restrictions","internal_movement_restrictions"
        ,"internal_movement_restrictions","information_campaigns","testing_policy"
        ,"contact_tracing","facial_coverings"
        ,"vaccination_policy","elderly_people_protection","population","cfr","cases_per_population","incidence"]]


input_scaler = MinMaxScaler(feature_range=(0, 1))
output_scaler = MinMaxScaler(feature_range=(0, 1))

predict_model = tf.keras.models.load_model("model.h5")

## Einlesen der Ein- und Ausgabeinfos des Models
INPUT_STEPS = predict_model.get_build_config()["input_shape"][1]
INPUT_COLS = predict_model.get_build_config()["input_shape"][2]
OUTPUT_STEPS = predict_model.get_config()["layers"][2]["config"]["n"]

SEQUENCE_COUNT = 1 # Wie viele Sequenzen (jeweils INPUT_STEPS lang und mit INPUT_COLS Spalten) werden eingegeben

selected_lk = "LK Ahrweiler"
lk_data_index = INPUT_STEPS*SEQUENCE_COUNT

## Einlesen der Daten
data_csv = pd.read_csv('../data/clean/df_weekly_incidence.csv')
lk_data_full= data_csv[data_csv.administrative_area_level_3 == selected_lk]

predict_year_weeks = lk_data_full[["year","week"]].iloc[lk_data_index:lk_data_index+OUTPUT_STEPS]


## Extrahieren der für das Netwerk wichtigen Spalten
lk_data = get_relevant_cols(lk_data_full)

## Normieren der Daten
lk_input_data = lk_data.to_numpy()[:,:-1]
lk_input_data = input_scaler.fit_transform(lk_input_data)

lk_output_data = lk_data.to_numpy()[:,-1].reshape(lk_data.shape[0],1)
output_scaler.fit(lk_output_data)


## Eingabesequenz extrahieren und reshapen 
data_to_predict = lk_input_data[0:lk_data_index,:]
data_to_predict = data_to_predict.reshape(SEQUENCE_COUNT,INPUT_STEPS,INPUT_COLS)

## Prediction 
prediction_result = predict_model.predict(data_to_predict,verbose=0).reshape(SEQUENCE_COUNT*OUTPUT_STEPS,1)
prediction_result = output_scaler.inverse_transform(prediction_result)[:,0]


result_df = pd.DataFrame(data={
    "Jahr":predict_year_weeks.year.to_numpy(),
    "Woche":predict_year_weeks.week.to_numpy(),
    "Inzidenz":prediction_result})

print("Vorhersage für '{}' ab Woche {} in {}".format(selected_lk,predict_year_weeks.week.iloc[0],predict_year_weeks.year.iloc[0]))
display(result_df)

Vorhersage für 'LK Ahrweiler' ab Woche 32 in 2020


,Jahr,Woche,Inzidenz
0,2020,32,8.273078
1,2020,33,9.777314
2,2020,34,8.625190
3,2020,35,9.102738
4,2020,36,9.918129
5,2020,37,12.049701
6,2020,38,16.034456
7,2020,39,21.451916
8,2020,40,26.963705
9,2020,41,32.056705


10

In [ ]:
a = np.array([[1,2],[0,3]])
b = np.array([[10,7],[5,13]])
s = MinMaxScaler(feature_range=(0,1)) 
a_s = s.fit_transform(a)
print(a_s)
b_s = s.transform(b)

print(b_s)

[[1. 0.]
 [0. 1.]]
[[10.  5.]
 [ 5. 11.]]
